## Домашнее задание «Продвинутый pandas 2»
### Задание 1

Используйте файл с оценками фильмов ml-latest-small/ratings.csv. 
Посчитайте среднее время жизни пользователей, которые выставили более 100 оценок. 
Под временем жизни понимается разница между максимальным и минимальным значением столбца timestamp для данного значения userId.

In [2]:
import pandas as pd

In [3]:
df = pd.read_csv('ml-latest-small/ratings.csv')
df.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [49]:
visits = df.groupby('userId').agg({'timestamp':['min', 'max', 'count']}).reset_index()
visits.iloc[:6]

userId   timestamp                  
                 min         max count
0      1  1260759108  1260759205    20
1      2   835355395   835356246    76
2      3  1298861589  1298932787    51
3      4   949778714   949982274   204
4      5  1163373044  1163375145   100
5      6  1108134263  1109258285    44

In [44]:
visits_filtred = visits[visits['timestamp']['count'] > 100]
visits_filtred.iloc[:4]

userId   timestamp                  
                  min         max count
3       4   949778714   949982274   204
7       8  1154389340  1154474527   116
14     15   997937239  1469330735  1700
16     17  1127468587  1127476640   363

In [54]:
# среднее время жизни каждого пользователя
visits_filtred['life_time'] = visits_filtred[('timestamp', 'max')] - visits_filtred[('timestamp', 'min')]
visits_filtred.iloc[:4]

userId   timestamp                    life_time
                  min         max count           
3       4   949778714   949982274   204     203560
7       8  1154389340  1154474527   116      85187
14     15   997937239  1469330735  1700  471393496
16     17  1127468587  1127476640   363       8053

In [81]:
avg_life = visits_filtred['life_time'].mean()
print(f'Среднее время жизни активных зрителей {int(avg_life)}')

Среднее время жизни активных зрителей 40080507


In [90]:
#timestamp Unixtime - Количество секунд, прошедших с 1 января 1970 года по UTC
print(f'{40080507 / 60 / 60 / 24} days')

463.8947569444444 days


In [22]:
from datetime import datetime

In [88]:
datetime.fromtimestamp(40080507)

datetime.datetime(1971, 4, 10, 0, 28, 27)

### Задание 2

Дана статистика услуг перевозок клиентов компании по типам (см. файл с кодом занятия). Необходимо сформировать две таблицы:
* таблицу с тремя типами выручки для каждого client_id без указания адреса клиента
* аналогичную таблицу по типам выручки с указанием адреса клиента

Обратите внимание, что в процессе объединения таблиц данные не должны теряться.
Дана статистика услуг перевозок клиентов компании по типам:

    rzd - железнодорожные перевозки
    auto - автомобильные перевозки
    air - воздушные перевозки
    client_base - адреса клиентов

In [91]:
rzd = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115],
        'rzd_revenue': [1093, 2810, 10283, 5774, 981]
    }
)
rzd

,client_id,rzd_revenue
0,111,1093
1,112,2810
2,113,10283
3,114,5774
4,115,981


In [92]:
auto = pd.DataFrame(
    {
        'client_id': [113, 114, 115, 116, 117],
        'auto_revenue': [57483, 83, 912, 4834, 98]
    }
)
auto

,client_id,auto_revenue
0,113,57483
1,114,83
2,115,912
3,116,4834
4,117,98


In [93]:
air = pd.DataFrame(
    {
        'client_id': [115, 116, 117, 118],
        'air_revenue': [81, 4, 13, 173]
    }
)
air

,client_id,air_revenue
0,115,81
1,116,4
2,117,13
3,118,173


In [94]:
client_base = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115, 116, 117, 118],
        'address': ['Комсомольская 4', 'Энтузиастов 8а', 'Левобережная 1а', 'Мира 14', 'ЗЖБИиДК 1', 
                    'Строителей 18', 'Панфиловская 33', 'Мастеркова 4']
    }
)
client_base

,client_id,address
0,111,Комсомольская 4
1,112,Энтузиастов 8а
2,113,Левобережная 1а
3,114,Мира 14
4,115,ЗЖБИиДК 1
5,116,Строителей 18
6,117,Панфиловская 33
7,118,Мастеркова 4


In [95]:
revenue_df = rzd.merge(auto, how='outer', on = 'client_id').merge(air, how='outer', on = 'client_id')
revenue_df

,client_id,rzd_revenue,auto_revenue,air_revenue
0,111,1093.0,NaN,NaN
1,112,2810.0,NaN,NaN
2,113,10283.0,57483.0,NaN
3,114,5774.0,83.0,NaN
4,115,981.0,912.0,81.0
5,116,NaN,4834.0,4.0
6,117,NaN,98.0,13.0
7,118,NaN,NaN,173.0


In [96]:
revenue_df_with_address = revenue_df.merge(client_base,  how='left', on = 'client_id')
revenue_df_with_address

,client_id,rzd_revenue,auto_revenue,air_revenue,address
0,111,1093.0,NaN,NaN,Комсомольская 4
1,112,2810.0,NaN,NaN,Энтузиастов 8а
2,113,10283.0,57483.0,NaN,Левобережная 1а
3,114,5774.0,83.0,NaN,Мира 14
4,115,981.0,912.0,81.0,ЗЖБИиДК 1
5,116,NaN,4834.0,4.0,Строителей 18
6,117,NaN,98.0,13.0,Панфиловская 33
7,118,NaN,NaN,173.0,Мастеркова 4


### Задание 3

В задаче сквозной аналитики вам предоставили данные по местоположению пользователей. 
Т.е. для каждого user_id известна последовательность координат (широта/долгота), когда они требовались приложению для полноценной работы. Как бы вы добавили эти сведения в таблицу визитов и покупок? Для составления ответа можно использовать вопросы:
* У каждого пользователя известен набор координат. А для связывания с визитом или фактом покупки скорее всего потребуется одно-два числа. Как их получить?
* Наборы координат одного и того же пользователя могут быть значительно удалены друг от друга. Как это отразится на вопросе расчетах пункта 1?
* Какие дополнительные признаки можно получить из координат? Ведь это просто числа, которые сами по себе мало что дают.

In [ ]:
# Ответ
К сожалению из условия не понятно, для чего используется приложение.

Можно определить скаляр координат (широты/долготы)

Как вариант можно попробовать представить координаты покупок и визитов в виде отдельных матриц.
При входе в приложение определять косинусное сходство между и предлагать популярные товары в данном регионе

Из координат также можно получить часовой пояс и город